In [1]:
import pandas as pd
import plotly.graph_objects as go
from signals import *

In [2]:
def analyse_signal(df, signal):
    """
    Process signal using SL and TP, assume a 1 minute delay to enter the trade.
    TODO: accound for spread and fees
    TODO: Warn if slippage is high
    """
    real_entry = None
    for dt, ohlc in df.loc[signal.datetime_str:].iterrows():
        if not real_entry:
            real_entry = ohlc.Close # Assume it takes 1 min to enter the trade
            
        if signal.type == "BUY":
            if ohlc.High >= signal.take_profit:
                return abs(signal.take_profit - real_entry)

            if ohlc.Low <= signal.stop_loss:
                return -1 * abs(real_entry - signal.stop_loss)
            
        if signal.type == "SELL":
            if ohlc.Low <= signal.take_profit:
                return abs(signal.take_profit - real_entry)

            if ohlc.High >= signal.stop_loss:
                return -1 * abs(real_entry - signal.stop_loss)
        
    return 0.0
        

In [3]:
beta_tp1 = Signal(ticker="AUDCAD", datetime_str="2020-01-14 13:19", 
                  entry_price=0.9020, stop_loss=0.9075, take_profit=0.9000)
df = load_ticker(beta_tp1.ticker)
print("TP1", analyse_signal(df, beta_tp1) * pip_factor(beta_tp1.ticker))

TP1 14.999999999999458


In [11]:
# TP1 Analysis using trade signals as given

signals = load_beta_trades("./beta/BETA.csv")
total_profit = 0.0
wins = 0
losses = 0
for signal in signals["TP1"]:
    df = load_ticker(signal.ticker)
    
    profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
    if profit > 0:
        wins += 1
    else:
        losses += 1
    total_profit += profit
    print(signal, profit)
print("Default:", total_profit, wins, losses)

AUDCAD: 2020-01-14 13:19: Entry: 0.902, SL: 0.9075, TP: 0.9 14.999999999999458
AUDNZD: 2020-01-14 13:16: Entry: 1.043, SL: 1.048, TP: 1.04 28.999999999999027
CADJPY: 2020-01-14 13:13: Entry: 84.2, SL: 84.85, TP: 84.0 15.999999999999659
AUDUSD: 2020-01-14 13:08: Entry: 0.69, SL: 0.6965, TP: 0.688 16.00000000000046
NZDJPY: 2020-01-14 13:06: Entry: 73.6, SL: 73.6, TP: 72.6 17.00000000000017
XAUUSD: 2020-01-14 10:56: Entry: 1544.94, SL: 1534.0, TP: 1546.94 24.400000000000546
GBPUSD: 2020-01-13 14:06: Entry: 1.29843, SL: 1.29, TP: 1.30043 21.299999999999653
XAUUSD: 2020-01-10 01:31: Entry: 1551.91, SL: 1558.0, TP: 1549.91 17.100000000000364
GBPJPY: 2020-01-09 02:08: Entry: 143.174, SL: 143.674, TP: 142.974 17.600000000001614
USDCAD: 2020-01-06 13:03: Entry: 1.2969, SL: 1.293, TP: 1.2989 20.999999999999908
NZDCAD: 2019-12-04 12:44: Entry: 0.8656, SL: 0.8715, TP: 0.8636 19.000000000000128
AUDJPY: 2019-12-04 12:43: Entry: 74.27, SL: 74.9, TP: 74.07 18.000000000000682
CADJPY: 2019-12-04 12:41: 

In [7]:
# TP1 Stop Loss analysis
signals = load_beta_trades("./beta/BETA.csv")
stops = range(20, 51, 1)
profits = []

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP1"]:
        signal.set_stop_loss_pips(stop)
        df = load_ticker(signal.ticker)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(stop, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title="BETA (Nov & Dec 2019 Signals) Stop Loss Optimisation for TP1",
    xaxis_title="Take Profit (pips)",
    yaxis_title="Profit (pips)",
    title_x=0.5
)
fig.show()


20 337.1999999999991 37 15
21 324.20000000000175 37 15
22 311.20000000000135 37 15
23 298.2000000000012 37 15
24 329.20000000000357 38 14
25 407.200000000005 40 12
26 453.1999999999991 41 11
27 444.2000000000025 41 11
28 435.20000000000164 41 11
29 426.2000000000008 41 11
30 417.2000000000042 41 11
31 408.19999999999663 41 11
32 504.30000000000024 43 9
33 497.29999999999916 43 9
34 490.2999999999981 43 9
35 538.300000000001 44 8
36 532.299999999997 44 8
37 526.2999999999965 44 8
38 520.2999999999987 44 8
39 514.2999999999981 44 8
40 508.29999999999757 44 8
41 563.299999999999 45 7
42 558.2999999999995 45 7
43 553.2999999999982 45 7
44 548.3000000000018 45 7
45 543.3000000000003 45 7
46 538.2999999999981 45 7
47 533.2999999999992 45 7
48 528.2999999999976 45 7
49 523.3000000000013 45 7
50 518.2999999999998 45 7


In [ ]:
# TP2 Stop loss analysis
signals = load_beta_trades("./beta/BETA.csv")

stops = range(22, 43, 1)
profits = []

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP2"]:
        signal.set_stop_loss_pips(stop)
        df = load_ticker(signal.ticker)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(stop, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title="BETA (Nov & Dec 2019 Signals) Stop Loss Optimisation for TP2",
    xaxis_title="Stop Loss (pips)",
    yaxis_title="Profit (pips)",
    title_x=0.5
)
fig.show()


In [8]:
# TP3 Stop loss analysis
signals = load_beta_trades("./beta/BETA.csv")

stops = range(20, 31, 1)
profits = []

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP3"]:
        signal.set_stop_loss_pips(stop)
        df = load_ticker(signal.ticker)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(stop, total_profit, wins, losses)
fig = go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title="BETA (Nov & Dec 2019 Signals) Stop Loss Optimisation for TP3",
    xaxis_title="Stop Loss (pips)",
    yaxis_title="Profit (pips)",
    title_x=0.5
)
fig.show()


20 681.9999999999944 15 37
21 649.0000000000059 15 37
22 723.0000000000003 16 36
23 690.9999999999975 16 36
24 907.0000000000058 18 34
25 877.0000000000039 18 34
26 873.9999999999915 18 34
27 844.9999999999984 18 34
28 815.9999999999967 18 34
29 786.9999999999925 18 34
30 758.0000000000019 18 34


In [20]:
# Take Profit analysis
signals = load_beta_trades("./beta/BETA.csv")

stop_loss = 34
take_profits = range(50, 81, 2)
profits = []

for take_profit in take_profits:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP3"]:
        signal.set_stop_loss_pips(stop_loss)
        signal.set_take_profit_pips(take_profit)
        df = load_ticker(signal.ticker)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(take_profit, total_profit, wins, losses)
    

50 921.9999999999839 32 20
52 896.000000000002 31 21
54 954.0000000000057 31 21
56 1012.0000000000051 31 21
58 1069.9999999999993 31 21
60 1127.9999999999902 31 21
62 1028.9999999999977 29 23
64 1082.9999999999893 29 23
66 1036.999999999988 28 24
68 1088.999999999995 28 24
70 1140.999999999986 28 24
72 1087.0000000000014 27 25
74 1028.999999999996 26 26
76 967.0000000000043 25 27
78 1012.9999999999997 25 27
80 944.9999999999943 24 28


In [21]:
fig = go.Figure(data=go.Scatter(x=list(take_profits), y=profits))
fig.update_layout(
    title="BETA (Nov & Dec 2019 Signals) Take Profit Optimisation (Stop Loss = 24 pips)",
    title_x=0.5,
    xaxis_title="Take Profit Level (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()

In [19]:
# Stop Loss analysis
signals = load_beta_trades("./beta/BETA.csv")

take_profit = 60
stops = range(20, 40, 2)
profits = []

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP3"]:
        signal.set_stop_loss_pips(stop)
        signal.set_take_profit_pips(take_profit)
        df = load_ticker(signal.ticker)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(stop, total_profit, wins, losses)
    

20 903.9999999999935 26 26
22 939.9999999999973 27 25
24 979.9999999999999 28 24
26 1109.99999999999 30 22
28 1071.999999999994 30 22
30 1033.9999999999984 30 22
32 1087.9999999999948 31 21
34 1127.9999999999902 31 21
36 1095.999999999987 31 21
38 1063.9999999999925 31 21


In [16]:
fig = go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title="BETA (Nov & Dec 2019 Signals) Stop Loss Optimisation (Take Profit = 59 pips)",
    title_x=0.5,
    xaxis_title="Stop Loss Level (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()

In [ ]:
# Example plotly candlestick plot

dfp = df.loc["2019-01-14 21:00":"2019-01-14 22:00"]
fig = go.Figure(data=[go.Candlestick(x=dfp.index,
                open=dfp['Open'],
                high=dfp['High'],
                low=dfp['Low'],
                close=dfp['Close'])])

fig.show()